# Replication of results from Anderson et al. 2016

In [1]:
import numpy as np
import scipy.io
import xarray as xr
import pyhsmm_mvpa as hsmm
import itertools
import multiprocessing 
import matplotlib.pyplot as plt


cpus = multiprocessing.cpu_count()

%load_ext autoreload
%autoreload 2


# Initial fit

Reading the data

In [2]:
mat = scipy.io.loadmat('matlab/newbumps/starter.mat')

data = np.array(mat['normedscore10'])
starts = np.array(mat['x'][:,0]) -1#correcting to be 0 indexed
ends = np.array(mat['y'][:,0])-1#correcting to be 0 indexed
subjects = np.array(mat['subjects'])-1 #correcting to be 0 indexed

Initializing the hsmm object with data, starts, sampling frequency... all things shared across the different following fit

In [3]:
init = hsmm.hsmm(data, starts, ends, sf=100, bump_width=50)

Estimating n_bump model with backward estimation

In [ ]:
bests = init.backward_estimation()
bests

Estimating parameters for 8 bumps model


In [ ]:
plt.plot(np.arange(init.max_bumps)+1, bests.likelihoods[::-1],'o-')
plt.show()
for nbump in np.arange(init.max_bumps,0,-1):
    times = init.mean_bump_times(bests.sel(n_bumps=nbump))
    plt.plot(times, -np.repeat(nbump, len(times)),'o')
